In [10]:
## version 2.1 Codigo Final para determina ruta optima a pie

##Codigo para determinar la ruta más corta pediante la api de Tom Tom

import requests
import geopandas as gpd
import folium
import json
import time
from IPython.display import display

# 📂 Cargar los archivos GeoJSON
#centroides = gpd.read_file("0-100.geojson") #cambiar capa de inicio
#centroides = gpd.read_file("101-200.geojson") #cambiar capa de inicio
#centroides = gpd.read_file("201-300.geojson") #cambiar capa de inicio
#centroides = gpd.read_file("301-400.geojson") #cambiar capa de inicio
#centroides = gpd.read_file("401-500.geojson") #cambiar capa de inicio
#centroides = gpd.read_file("501-600.geojson") #cambiar capa de inicio
#centroides = gpd.read_file("601-700.geojson") #cambiar capa de inicio
#centroides = gpd.read_file("701-800.geojson") #cambiar capa de inicio
#centroides = gpd.read_file("801-900.geojson") #cambiar capa de inicio
centroides = gpd.read_file("901-983.geojson") #cambiar capa de inicio
#playa = gpd.read_file("playa_carvallo.geojson") # cambiar capa destino
#playa = gpd.read_file("playa_san_mateo.geojson") # cambiar capa destino
playa = gpd.read_file("sector_ruben_dario.geojson") # cambiar capa destino

# 📍 Asegurar que ambos archivos están en coordenadas WGS84 (EPSG:4326)
centroides = centroides.to_crs(epsg=4326) if centroides.crs != "EPSG:4326" else centroides
playa = playa.to_crs(epsg=4326) if playa.crs != "EPSG:4326" else playa

# Convertir MULTIPOINT a POINT tomando el primer punto de cada geometría
centroides["geometry"] = centroides["geometry"].apply(lambda geom: geom.geoms[0] if geom.geom_type == "MultiPoint" else geom)

# Verificar si se convirtieron correctamente
print(centroides.head())


# 📌 Seleccionar 80 puntos aleatorios de Centroide_2
#centroides = centroides.sample(80, random_state=42)

# 📍 Obtener la coordenada del destino (playa)
destino = playa.geometry.centroid.iloc[0]
destino_coords = (destino.y, destino.x)  # (lat, lon)

# 🗺 Crear un mapa interactivo en Jupyter Notebook
api_key = "sUyayTkWsvenYJ3oaNC9WFuxYsPPJlIA"
mapa = folium.Map(
    location=destino_coords,
    zoom_start=12,
    tiles=f"https://api.tomtom.com/map/1/tile/basic/main/{{z}}/{{x}}/{{y}}.png?view=Unified&key={api_key}",
    attr="TomTom"
)

# 🚀 API de TomTom para calcular rutas
url_base = "https://api.tomtom.com/routing/1/calculateRoute"
headers = {"Content-Type": "application/json"}

# 📂 Lista para almacenar rutas en GeoJSON
rutas_10_geojson = []

# 📌 Iterar sobre los 80 puntos seleccionados y calcular rutas
for idx, row in centroides.iterrows():
    origen = row.geometry
    if origen.geom_type == "Point":
        inicio_coords = (origen.y, origen.x)  # (lat, lon)
        
        url = f"{url_base}/{inicio_coords[0]},{inicio_coords[1]}:{destino_coords[0]},{destino_coords[1]}/json"
        params = {
            "key": api_key,
            "travelMode": "pedestrian",
            "routeType": "fastest"
        }
        
        try:
            response = requests.get(url, params=params, headers=headers)
            if response.status_code == 200:
                data = response.json()
                route = data["routes"][0]["legs"][0]["points"]
                route_coords = [(point["latitude"], point["longitude"]) for point in route]
                
                # 📍 Unir la línea con los puntos: agregar el punto de inicio y destino
                route_coords = [inicio_coords] + route_coords + [destino_coords]
                
                # 📍 Dibujar la ruta en el mapa
                folium.PolyLine(route_coords, color="blue", weight=2, opacity=0.6).add_to(mapa)
                
                # 📂 Guardar la ruta en GeoJSON
                rutas_10_geojson.append({
                    "type": "Feature",
                    "geometry": {
                        "type": "LineString",
                        "coordinates": [[lon, lat] for lat, lon in route_coords]
                    },
                    "properties": {
                        "origen": inicio_coords,
                        "destino": destino_coords
                    }
                })
                
                # ⚡ Evitar demasiadas solicitudes seguidas
                time.sleep(1)

            else:
                print(f"Error en el punto {idx}: {response.status_code}")

        except Exception as e:
            print(f"Error en el punto {idx}: {e}")

# 📂 Guardar el GeoJSON con todas las rutas
geojson_data = {
    "type": "FeatureCollection",
    "features": rutas_10_geojson
}

with open("rutas_10.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson_data, f, ensure_ascii=False, indent=4)

# 📌 Agregar marcadores de inicio y destino al mapa
folium.Marker(destino_coords, popup="Playa Carvallo", icon=folium.Icon(color="red")).add_to(mapa)
for idx, row in centroides.iterrows():
    origen = row.geometry
    if origen.geom_type == "Point":
        folium.CircleMarker(
            location=[origen.y, origen.x],
            radius=3,
            color="blue",
            fill=True,
            fill_color="blue",
            fill_opacity=0.6
        ).add_to(mapa)

# 🖥 Mostrar el mapa en Jupyter Notebook
display(mapa)

                DESTINO    FID      ROL_SII  Superficie   ID  \
0          Habitacional  24313  01003-00022      165.04  901   
1          Habitacional  23760  01043-00028      127.53  902   
2          Habitacional  24059  01056-00017      196.29  903   
3          Habitacional  24343  01068-00014      172.08  904   
4  Deporte y Recreación  32050  01073-00002    44027.60  905   

                      geometry  
0  POINT (-71.63087 -33.02872)  
1  POINT (-71.63448 -33.02427)  
2  POINT (-71.63687 -33.02778)  
3  POINT (-71.63758 -33.02532)  
4  POINT (-71.63868 -33.02328)  


C:\Users\jose.cordova\AppData\Local\Temp\ipykernel_26776\181862986.py:40: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  destino = playa.geometry.centroid.iloc[0]


               DESTINO    FID      ROL_SII  Superficie  ID  \
0         Habitacional  24225  01037-00014      545.98   1   
1         Habitacional  24813  01008-00007      546.74   2   
2               Bodega   3841  01087-00449     2801.44   3   
3  Educación y Cultura  32107  01045-00002       53.13   4   
4         Habitacional  25199  01020-00008      463.56   5   

                         geometry  
0  POINT (253929.319 6342504.498)  
1  POINT (254212.575 6341985.531)  
2  POINT (253158.578 6343013.586)  
3   POINT (253941.996 6343013.51)  
4   POINT (254171.73 6341978.941)  
